Actividad Limpieza de base de datos - Bike buyers

Mario Alberto Perez Barrera - A01799928

Marco Antonio Gonzalez Fernandez - A01799122

ETAPA DE INSPECCION

In [33]:
import pandas as pd
import numpy as np


# Cargar el archivo CSV
df = pd.read_csv('bike_buyers.csv')

In [34]:
# Inspección inicial
print("Primeras filas del conjunto de datos:")
df.head()

Primeras filas del conjunto de datos:


,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12591,Married,Female,"30,000",4.0,Graduate Degree,Clerical,Yes,0.0,0-1 Miles,Europe,45.0,No
1,14058,Single,Male,"70,000",0.0,Bachelors,Professional,No,1.0,5-10 Miles,Pacific,41.0,Yes
2,23962,Married,Female,"10,000",0.0,Partial High School,Manual,Yes,2.0,1-2 Miles,Europe,32.0,No
3,23419,Single,Female,"70,000",5.0,Bachelors,Professional,Yes,3.0,10+ Miles,Pacific,39.0,No
4,21940,Married,Male,"90,000",5.0,Graduate Degree,Professional,Yes,0.0,0-1 Miles,North America,47.0,Yes


In [35]:
#Lectura de la informacion de la base de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1251 entries, 0 to 1250
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID                1251 non-null   int64  
 1   Marital Status    1244 non-null   object 
 2   Gender            1238 non-null   object 
 3   Income            1243 non-null   object 
 4   Children          1238 non-null   float64
 5   Education         1251 non-null   object 
 6   Occupation        1251 non-null   object 
 7   Home Owner        1247 non-null   object 
 8   Cars              1242 non-null   float64
 9   Commute Distance  1251 non-null   object 
 10  Region            1251 non-null   object 
 11  Age               1238 non-null   float64
 12  Purchased Bike    1251 non-null   object 
dtypes: float64(3), int64(1), object(9)
memory usage: 127.2+ KB


In [36]:
#Resumen estadístico de un DataFrame.
df.describe()

,ID,Children,Cars,Age
count,1251.000000,1238.000000,1242.000000,1238.000000
mean,20030.208633,1.929725,1.479066,44.058966
std,5331.451777,1.638977,1.121885,11.271138
min,11000.000000,0.000000,0.000000,25.000000
25%,15465.000000,0.000000,1.000000,35.000000
50%,19731.000000,2.000000,1.000000,43.000000
75%,24549.000000,3.000000,2.000000,52.000000
max,29447.000000,5.000000,4.000000,89.000000


In [37]:
# Revisión de valores faltantes
print(df.isnull().sum())

ID                   0
Marital Status       7
Gender              13
Income               8
Children            13
Education            0
Occupation           0
Home Owner           4
Cars                 9
Commute Distance     0
Region               0
Age                 13
Purchased Bike       0
dtype: int64


ETAPA DE LIMPIEZA